In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import json

f = open('/content/tag2idx')
tag2idx = json.load(f)
# Closing file
f.close()

all_tags_list = list(tag2idx.keys())

In [ ]:
import pandas as pd

full_train_df = pd.read_csv('/content/Final_DataFrame_Train_Preamble')

train_sents = full_train_df['clean_sent'].to_list()
train_labels = full_train_df['tags'].to_list()

train_sents = [train_sents[i].replace("'", "").strip(
    '][').split(', ') for i in range(len(train_sents))]
train_labels = [train_labels[i].replace("'", "").strip(
    '][').split(', ') for i in range(len(train_labels))]


print(len(train_sents[0]))
print(len(train_labels[0]))

print(len(train_sents))
print(len(train_labels))

print(train_sents[0])
print(train_labels[0])

In [ ]:
new_df_list = []

for i in range(len(train_sents)):
  for word, tag in zip(train_sents[i], train_labels[i]):
    row = [str(i), word, tag]
    new_df_list.append(row)

print(new_df_list)

In [ ]:
import pandas as pd

train_df = pd.DataFrame(new_df_list, columns=['sentence_id', 'words', 'labels'])
# train_df.rename(columns={'tags': 'labels'}, inplace=True)
train_df.head(-1)

print(train_df.isnull().values.any())

In [ ]:
full_test_df = pd.read_csv('/content/Final_DataFrame_Dev_Preamble')

test_sents = full_test_df['clean_sent'].to_list()
test_labels = full_test_df['tags'].to_list()

test_sents = [test_sents[i].replace("'", "").strip(
    '][').split(', ') for i in range(len(test_sents))]
test_labels = [test_labels[i].replace("'", "").strip(
    '][').split(', ') for i in range(len(test_labels))]


print(len(test_sents[0]))
print(len(test_labels[0]))

print(len(test_sents))
print(len(test_labels))

print(test_sents[0])
print(test_labels[0])

In [ ]:
new_df1_list = []

for i in range(len(test_sents)):
  for word, tag in zip(test_sents[i], test_labels[i]):
    row = [str(i), word, tag]
    new_df1_list.append(row)

print(new_df1_list)

In [ ]:
# print(len(train_df))
# train_df_drop = train_df.dropna()
# print(len(train_df_drop))

In [ ]:
import pandas as pd

test_df = pd.DataFrame(new_df1_list, columns=['sentence_id', 'words', 'labels'])
# train_df.rename(columns={'tags': 'labels'}, inplace=True)
test_df.head(-1)

In [ ]:
# print(len(test_df))
# test_df = test_df.dropna()
# print(len(test_df))

In [ ]:
!pip install simpletransformers

In [ ]:
device = 'cuda'
import torch, gc
import os
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from simpletransformers.ner import NERModel, NERArgs
import sklearn


model_args = NERArgs()
model_args.labels_list = all_tags_list
model_args.train_batch_size = 10
model_args.num_train_epochs = 10
model_args.classification_report = True

model = NERModel(
  "bert",
  "bert-base-cased",
  args=model_args,
  use_cuda=True,
)

In [ ]:
all_tags_list

In [ ]:
model.train_model(train_df)

In [ ]:
result, model_outputs, preds_list = model.eval_model(test_df)


In [ ]:
print(preds_list)

In [ ]:
len(preds_list[0])

In [ ]:
print(result)

In [ ]:
predictions, raw_outputs = model.predict(["Before The Madurai Bench Of Madras High Court\n\nDated: 23/12/2011\n\nCoram\nThe Honourable Mr.Justice V.Ramasubramanian\n\nCivil Revision Petition (Npd)(Md) No.1123 of 2006\nAnd\nM.P.No.2 of 2006\n\n1. Ayisha Beevi\n2. Beevija\n3. Hadijath Beevi\n4. Yunusa Begam\n5. Syed Ali\n6. Sumaya Begam\n7. Mohamed Yoosuf\n8. Mohamed Ismail\n9. Razira Beevi\n10.Shabi Mohamed\n11.Zakir Mugain\n12.Ferosh Khan\n13.Augustin\n14.Dr.T.C.Joseph\t\t\t..... Petitioners\n\nVs.\n\n1. Sheik Mydeen\n2. A.P.Nelson\n3. Chandrakala Ruben\t\t\t..... Respondents\n-----\n\tPetition under Article 227 of  the Constitution of India against the fair\nand decretal order dated 18.10.2006 made in E.P.No.201 of 2000 in O.S.No.420 of\n1976 passed by the Principal District Munsif, Padmanabhapuram.\n\n!For Petitioner \t    \t ...   Mr.K.N.Thampi\n^For Respondent-1 \t  \t ...   Mr.Prabhu Rajadurai\nFor Respondents 2&3 \t\t ...   Mr.A.Arumugham\n-----\n:Order"])

In [ ]:
print(predictions[0])

In [ ]:
print(preds_list[0])

In [ ]:
test_df['labels']

In [ ]:
valid_examples = []
count = 0
for i,j in list(zip(train_labels, preds_list)):
  if len(i) == len(j):
    valid_examples.append(count)
  count += 1

In [ ]:
valid_examples

In [ ]:
from sklearn.metrics import classification_report


print(classification_report(train_labels[0], preds_list[0]))

### Clubbing Labels


In [ ]:
def club_IOB_labels(labels):
  club_labels = []
  for label_list in labels:
    club_label_list = []
    for label in label_list:
      if label != 'O':
        strip_label = label.split('-')[1]
        club_label_list.append(strip_label)
      else:
        club_label_list.append(label)
    club_labels.append(club_label_list)
  return club_labels

In [ ]:
clubbed_true_labels = club_IOB_labels(train_labels)

clubbed_pred_labels = club_IOB_labels(preds_list)

In [ ]:
print(classification_report(clubbed_true_labels[3], clubbed_pred_labels[3]))